In [26]:
import utils
from order_book import OrderBook
from trade import Trade, TradeHistory, history

import plotly.express as px
import polars as pl
import numpy as np

In [2]:
history.trades

[]

## Question 1 : simulating fair prices

In [3]:
# px.line(simulate_fair_price().to_pandas().set_index('Timestamp'))

Question for teacher : what is the economic reality behind these models / what are some good papers explaining these choices
- for a currency, going to 0.01 for a few months and then 50 months later is not realistic / signifies huge macroeconomic events
- what are some papers indicating which parameters make sense in this context

## Question 2 : Order book

In [4]:
# Example Usage
order_book = OrderBook(n_levels=5)
order_book.update_order(price=100.5, size=10, side="bid")
order_book.update_order(price=101.0, size=15, side="bid")
order_book.update_order(price=102.0, size=5, side="ask")
order_book.update_order(price=102.5, size=20, side="ask")
order_book.update_order(price=103.0, size=8, side="ask")

print("Best Bid:", order_book.get_best_bid())  # (101.0, 15)
print("Best Ask:", order_book.get_best_ask())  # (102.0, 5)
print("Order Book:\n", order_book.get_order_book())

Best Bid: (101.0, 15.0, datetime.time(8, 6, 41, 69850), False)
Best Ask: (102.0, 5.0, datetime.time(8, 6, 41, 75588), False)
Order Book:
 shape: (3, 8)
┌────────────┬─────────────────┬──────────┬───────┬───────┬──────────┬────────────────┬────────────┐
│ client_bid ┆ timestamp_bid   ┆ size_bid ┆ bid   ┆ ask   ┆ size_ask ┆ timestamp_ask  ┆ client_ask │
│ ---        ┆ ---             ┆ ---      ┆ ---   ┆ ---   ┆ ---      ┆ ---            ┆ ---        │
│ bool       ┆ time            ┆ f64      ┆ f64   ┆ f64   ┆ f64      ┆ time           ┆ bool       │
╞════════════╪═════════════════╪══════════╪═══════╪═══════╪══════════╪════════════════╪════════════╡
│ false      ┆ 08:06:41.069850 ┆ 15.0     ┆ 101.0 ┆ 102.0 ┆ 5.0      ┆ 08:06:41.07558 ┆ false      │
│            ┆                 ┆          ┆       ┆       ┆          ┆ 8              ┆            │
│ false      ┆ 08:06:40.975460 ┆ 10.0     ┆ 100.5 ┆ 102.5 ┆ 20.0     ┆ 08:06:41.07656 ┆ false      │
│            ┆                 ┆        

In [5]:
order_book.bids.bottom_k(k=1, by='bid').row(index=0) #gives us the worst available bid

(100.5, 10.0, datetime.time(8, 6, 40, 975460), False)

In [6]:
order_book.delete_order(price=102.0, size=5, side='ask')

## Question 4 : Trade logic

In [7]:
trade = Trade(26, "buy")

In [10]:
order_book.get_order_book()

client_bid,timestamp_bid,size_bid,bid,ask,size_ask,timestamp_ask,client_ask
bool,time,f64,f64,f64,f64,time,bool
false,08:06:41.069850,15.0,101.0,102.5,20.0,08:06:41.076562,false
false,08:06:40.975460,10.0,100.5,103.0,8.0,08:06:41.077761,false


In [11]:
ob = trade.update_orderbook_with_trade(orderbook=order_book)

In [12]:
ob.get_order_book()

client_bid,timestamp_bid,size_bid,bid,ask,size_ask,timestamp_ask,client_ask
bool,time,f64,f64,f64,f64,time,bool
false,08:06:41.069850,15.0,101.0,103.0,2.0,08:08:59.880288,false
false,08:06:40.975460,10.0,100.5,null,null,null,null


In [24]:
from datetime import date
pl.DataFrame(history.trades).filter(pl.col('timestamp').dt.date().eq(date.today()))

timestamp,side,price,size,client
datetime[μs],str,f64,f64,bool
2025-04-19 08:08:59.880288,"""buy""",103.0,6.0,false
2025-04-19 08:08:59.880288,"""buy""",102.5,6.0,false


In [15]:
from values import VOLUME_LIST

In [18]:
sum(VOLUME_LIST)

10000000

In [29]:
bid_asks = utils.compute_all_bid_ask(historical_fair_price=utils.simulate_fair_price(), volume=sum(VOLUME_LIST))

In [31]:
bid_asks.with_columns(pl.Series(name='orderbooks', values=[OrderBook(6)]*bid_asks.height))

timestamp,fair_price,log_return,volatility,spread,bid,ask,orderbooks
datetime[μs],f64,f64,f64,f64,f64,f64,object
2025-01-01 00:00:00,100.0,null,null,null,null,null,<order_book.OrderBook object at 0x0000023AE552CC10>
2025-01-01 01:00:00,100.151337,0.001512,0.0,0.0,100.151337,100.151337,<order_book.OrderBook object at 0x0000023AE552CC10>
2025-01-01 02:00:00,100.119779,-0.000315,0.000914,0.001827,100.117951,100.121606,<order_book.OrderBook object at 0x0000023AE552CC10>
2025-01-01 03:00:00,100.009353,-0.001104,0.001096,0.002191,100.007162,100.011544,<order_book.OrderBook object at 0x0000023AE552CC10>
2025-01-01 04:00:00,99.833615,-0.001759,0.001225,0.00245,99.831165,99.836066,<order_book.OrderBook object at 0x0000023AE552CC10>
…,…,…,…,…,…,…,…
2025-12-31 20:00:00,222.526111,0.000371,0.001064,0.002128,222.523983,222.528239,<order_book.OrderBook object at 0x0000023AE552CC10>
2025-12-31 21:00:00,222.470272,-0.000251,0.001064,0.002128,222.468145,222.4724,<order_book.OrderBook object at 0x0000023AE552CC10>
2025-12-31 22:00:00,222.086466,-0.001727,0.001064,0.002128,222.084338,222.088594,<order_book.OrderBook object at 0x0000023AE552CC10>
